# lotlist_origin作成

# ライブラリインポート

In [1]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import mojimoji
import shutil

# Step_1~4 eqdataの読み込み・加工

In [2]:
"""
eqdataの更新時間からデータの取得時間を算出する

"""

'\neqdataの更新時間からデータの取得時間を算出する\n\n'

## Step_1 eqdataからspeed > 3490のみ抽出

In [3]:
def read_csv(input_dir,output_dir,st_time,ed_time):
    
    files = glob.glob(input_dir)
    print(str(len(files))+"files")
    print("step_1 is started")  
    for file,i in zip(files,range(len(files))):
        out_file = output_dir+str(str(i).zfill(5))+".csv"
        
    ## ファイルの更新日時から時間のカラムを追加
        t = os.path.getmtime(file)
        d = datetime.datetime.fromtimestamp(t)
        if (np.datetime64(d)>=st_time) and (np.datetime64(d)<=ed_time):
            df = pd.read_csv(file,header=[2,3],index_col=0)
            df.insert(loc = 0, column= "file_end_time", value= str(d))
            df.columns = ["file_end_time","Time_msec","X_POS_mm","Z_POS_mm","X_TCMD_%","Z_TCMD_%","B_TCMD_%","C_TCMD_%","S_TCMD_%","S_SPEED_rpm"]
            df.insert(loc = 0, column= "file_start_time", value= str(d-(pd.to_timedelta(df["Time_msec"].astype(int).max(),"S"))/1000))
            
    ## speed>3490で加工中のみを抽出
            df=df[df["S_SPEED_rpm"]>=3490.0]
            df.to_csv(out_file, header=True, index=False, encoding="cp932")
        else:
            continue
    print("step_1 is finished")

## Step_2　抽出したcsvを結合

In [4]:
def merge_csv_files(output_dir):   
    
    print("step_2 is started")
    files = glob.glob(output_dir+"*.csv")
    dfs = []
    for file in files:
        i = pd.read_csv(file,encoding="cp932")
        dfs.append(i)
    df = pd.concat(dfs, axis=0)
    print("step_2 is finished")
    return df

## Step_3　uidの付与

In [5]:
def preprocess_data(df_servo):
    
    print("step_3 is started")
    df_servo["DATETIME"] = pd.to_datetime(df_servo["file_start_time"]) + pd.to_timedelta(df_servo["Time_msec"]/1000, "S")
    df_servo = df_servo.sort_values(["DATETIME"])
    
    count = 0
    uid = 0
    time_list = df_servo["DATETIME"]
    uid_list = []
    th_time = np.timedelta64(60, 's')

    for i in time_list:
        if count == 0:
            uid_list.append(uid)
            count += 1
            j = i
        else:
            if i > j + th_time:
                uid += 1
                uid_list.append(uid)
                j = i
            else:
                uid_list.append(uid)
                j = i

    df_servo["uid"] = uid_list
    
    print("step_3 is finished")

    return df_servo

## Step_4　FLAGの付与（0=製品加工,1=ドレッシング)

In [6]:
def create_df_flag(df_servo):  
    
    print("step_4 is started")
    df_uid = df_servo.loc[:,["uid","DATETIME"]].drop_duplicates(subset = "uid")
    #pandasのバージョン？で動きませんでした。↓に変更     df_std = df_servo.groupby("uid").std(numeric_only=True)
    # df_std = df_servo.groupby("uid").std(ddof =True)
    df_std = df_servo.groupby("uid").std(numeric_only=True, ddof=True)
    df_std.reset_index(inplace=True)
    df_etd = df_servo.groupby("uid").max()
    df_etd.reset_index(inplace=True)
    endtime = df_etd.loc[:,["uid","DATETIME"]]
    endtime = endtime.rename(columns={"DATETIME": "DATETIME_e"})
    df_std["FLAG"] = 0
    df_std.loc[df_std["B_TCMD_%"] <= 0.2, "FLAG"] = 1
    df_flag = df_uid.merge(df_std.loc[:,["uid","FLAG"]],how="inner",on="uid")
    df_flag = endtime.merge(df_flag.loc[:,["uid","DATETIME","FLAG"]],how="inner",on="uid")
    df_flag = df_flag.reindex(columns=['uid', 'DATETIME', 'DATETIME_e','FLAG'])
    df_flag = df_flag.rename(columns={'DATETIME':'DATETIME_S','DATETIME_e':'DATETIME_E'})
    print("step_4 is finished")
    


    return df_flag

# Step_5 ドレッシング記録の読み込み・加工

In [7]:
def create_df_dress(dre_dir): 
    print("step_5 is started")
    df_dre = pd.read_excel(dre_dir, skiprows=1)

    #"« NULL »"の文字列をnullに変更
    df_dre = df_dre.replace("« NULL »",np.nan)

    #typeの変更
    df_dre["s1_開始dt"] = pd.to_datetime(df_dre["s1_開始dt"])
    df_dre["s1_終了dt"] = pd.to_datetime(df_dre["s1_終了dt"])
    df_dre["s1_開始日付手入力"] = pd.to_datetime(df_dre["s1_開始日付手入力"])
    df_dre["s1_終了日付手入力"] = pd.to_datetime(df_dre["s1_終了日付手入力"])
    df_dre["s1_開始時間手入力"] = pd.to_datetime(df_dre["s1_開始時間手入力"])
    df_dre["s1_終了時間手入力"] = pd.to_datetime(df_dre["s1_終了時間手入力"])

    #開始dt ot終了dtにnatがあれば、手入力の時間で補完する。
    a = df_dre[['s1_開始dt','s1_終了dt','s1_開始日付手入力','s1_終了日付手入力','s1_開始時間手入力','s1_終了時間手入力','entry_timestamp']]
    df_dress = df_dre.loc[:,['s1_作業指図書no','s1_数量','s1_ドレスサイクルs', 's1_ドレスサイクルe','s1_開始dt', 's1_終了dt']]

    st_day = a["s1_開始日付手入力"].dt.strftime('%Y-%m-%d')
    st_time = a["s1_開始時間手入力"].dt.strftime('%H:%M:%S')
    st = st_day +" "+ st_time
    st = pd.to_datetime(st)

    et_day = a["s1_終了日付手入力"].dt.strftime('%Y-%m-%d')
    et_time = a["s1_終了時間手入力"].dt.strftime('%H:%M:%S')
    et = et_day +" "+ et_time

    b = pd.DataFrame(st,columns=['s1_手入力開始dt'])
    c = pd.DataFrame(et,columns=['s1_手入力終了dt'])
    df_dress = pd.concat([df_dress,b,c],axis=1)

    df_dress["s1_開始dt"].fillna(df_dress["s1_手入力開始dt"], inplace=True)
    df_dress["s1_終了dt"].fillna(df_dress["s1_手入力終了dt"], inplace=True)
    df_dress = df_dress.drop(["s1_手入力開始dt","s1_手入力終了dt"],axis=1)
    df_dress.to_excel(r"C:\Users\00220401626\Desktop\必要なファイル\Data_1\Output\3\ドレシング記録.xlsx",index=False)
    
    print("step_5 is finished")
        
    return df_dress

# Step_6 ドレッシング記録の結合

In [8]:
def flag_lots(df_flag, df_dress): 
    
    print("step_6 is started")
    df_flag["lotno"] = "0000000-000"

    for k in range(len(df_flag)):
        for i in range(len(df_dress)):
            fdt = np.datetime64(str(df_flag["DATETIME_S"][k]))
            sdt = np.datetime64(str(df_dress["s1_開始dt"][i]))
            edt = np.datetime64(str(df_dress["s1_終了dt"][i]))
            if (fdt >= sdt ) and (fdt <= edt ) :
                df_flag.loc[k, "lotno"] = df_dress.loc[i, "s1_作業指図書no"]
            elif (sdt > fdt ):
                break
    df_flag["lotno"] = df_flag["lotno"].apply(lambda x: mojimoji.zen_to_han(x) )
    print("step_6 is finished")
    
    return df_flag

# Step_7 focasdata読み込み・加工

In [9]:
def process_data_1(focas_dir1, df_flag):
    
    print("step_7 is started")
    df_focas = pd.read_csv(focas_dir1,sep="\t")
    # df_focas2 = pd.read_csv(focas_dir2,sep="\t")
    # df_focas = pd.concat([df_focas1, df_focas2], axis=0)
    
    df_focas['DATETIME'] = pd.to_datetime(df_focas['DATETIME'])
    df_flag["DATETIME_S"] = df_flag['DATETIME_S'].dt.round("s")
    df_flag["DATETIME_E"] = df_flag['DATETIME_E'].dt.round("s")
    df_focas['DATETIME_S'] = df_focas['DATETIME'].dt.round("s")
    df_focas.drop_duplicates(subset=["DATETIME"],keep='last',inplace=True)
    df_focas.pop('DATETIME')

    pu_cols = ['DATETIME_S','2:var818','2:x_axis', '2:z_axis']
    df_flag = df_flag.merge(df_focas.loc[:,pu_cols],left_on=["DATETIME_S"],right_on=["DATETIME_S"],how="inner")
    df_flag = df_flag.loc[:,["uid","FLAG","lotno","DATETIME_S","DATETIME_E",'2:var818','2:x_axis', '2:z_axis']]
    print("step_7 is finished")
    
    return df_flag

# Main_File

In [10]:
if __name__ == "__main__":
    ##製品の加工時期（範囲の入っていればよいです）
    start_time = "2023-02-07"
    end_time = "2023-02-08"
  
    st_time = np.datetime64(str(start_time))
    ed_time = np.datetime64(str(end_time))
    
    input_dir = r"C:\Users\00220401626\Desktop\必要なファイル\Data_1\eqdata\*.csv"
    focus_dir = r"C:\Users\00220401626\Desktop\必要なファイル\Data_1\1675728784.2723718.tsv" 
  
    dre_dir = r"C:\Users\00220401626\Desktop\必要なファイル\Data_1\ドレッシング記録20230216.xlsx"  
    output_dir = r"C:\Users\00220401626\Desktop\必要なファイル\Data_1\Output\1\\"

    
    read_csv(input_dir, output_dir, st_time, ed_time)         #step1　eqdataの読み込み・加工
    df=merge_csv_files(output_dir)                            #step2
    df_servo1=preprocess_data(df)                             #step3
    df_flag = create_df_flag(df_servo1)                       #step4
    df_dress = create_df_dress(dre_dir)                       #step5 ドレッシング記録の読み込み・加工
    df1 = flag_lots(df_flag, df_dress)                        #step6 df_dressとdf_flag結合
    df2 = process_data_1(focus_dir,df1)                       #step7 focasdataを読み込んでdf1と結合
    

280files
step_1 is started
step_1 is finished
step_2 is started
step_2 is finished
step_3 is started
step_3 is finished
step_4 is started
step_4 is finished
step_5 is started
step_5 is finished
step_6 is started
step_6 is finished
step_7 is started
step_7 is finished


In [11]:
df2

,uid,FLAG,lotno,DATETIME_S,DATETIME_E,2:var818,2:x_axis,2:z_axis
0,0,0,2300002-000,2023-02-07 10:24:57,2023-02-07 10:32:37,0.140,-219.386307,-94.583504
1,1,0,2300002-000,2023-02-07 10:36:38,2023-02-07 10:44:14,0.180,-219.406296,-104.206596
2,2,0,2300002-000,2023-02-07 10:50:26,2023-02-07 10:58:07,0.190,-219.411301,-99.313301
3,3,0,2300002-000,2023-02-07 12:15:42,2023-02-07 12:23:25,0.190,-219.410797,-96.764999
4,3,0,2300002-000,2023-02-07 12:15:42,2023-02-07 12:23:25,0.190,-219.410797,-98.151604
5,4,0,2300002-000,2023-02-07 12:36:16,2023-02-07 12:43:56,0.195,-219.412796,-94.984001
6,5,0,2300002-000,2023-02-07 12:51:21,2023-02-07 12:59:01,0.195,-219.413101,-93.884300
7,6,0,2300002-000,2023-02-07 13:11:43,2023-02-07 13:19:25,0.195,-219.413193,-92.241203
8,7,0,2300002-000,2023-02-07 13:29:11,2023-02-07 13:36:54,0.200,-219.415405,-95.118301
9,7,0,2300002-000,2023-02-07 13:29:11,2023-02-07 13:36:54,0.200,-219.415405,-96.504898


# CSV出力

In [12]:
df2.to_csv(r"C:\Users\00220401626\Desktop\必要なファイル\Data_1\Output\2\lotlist_origin.csv",index=False,encoding="cp932")